In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib notebook

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from functools import lru_cache, reduce
from zwad.ad import ZtfAnomalyDetector


algos_for_fields = {
    'm31': ['iso', 'gmm', 'svm', 'lof'],
    'deep': ['iso', 'gmm', 'svm'],
    'disk': ['iso', 'gmm'],
}

jobs = '4'
data_dir = os.path.join('..', 'data')
fake_dir = os.path.join(data_dir, 'fakes')
scalings = ['std', 'pca', 'pca15', 'pca24', 'norm']


# Contents

* [Code](#Code)
* [Generating score files with different scalings](#Generating-score-files-with-scalings)
* [Compare fake detection with different scalings](#Compare-fake-detection-with-scalings)
* [Generating score files for different fields](#Generating-score-files-for-different-fields)
* [Plot the fake detection curves for fields](#Plot-the-fake-detection-curves-for-fields)
* [Plot the cumulative score distributions](#Plot-the-cumulative-score-distributions)

# Code

In [3]:
@lru_cache()
def load_fake_names(fake_filename):
    """
    Just load the fake names, in tuple.
    """
    return tuple(pd.read_csv(fake_filename, index_col=0)['0'])


def fake_indices(scores, fake_names):
    """
    Calculate fake indices
    
    Parameters
    ----------
    scores: Scores of all the objects, including fakes at the end.
    
    fake_names: Tuple of fake object names.
    
    Return
    ------
    Table with 'name' and 'order' columns, sorted by 'order'.
    """
    fake_n = len(fake_names)
    index = np.argsort(scores)
    fake_index = np.argsort(index)[-fake_n:]  # Guess what's going on here ;)
    fake_table = pd.DataFrame({'order': fake_index, 'name': fake_names})
    return fake_table.sort_values(by='order').reset_index(drop=True)


def union_fakes(algo_to_fakes):
    """
    Union the different algorithms' fake detection curves to one
    """
    
    order = []
    for fake_table in algo_to_fakes.values():
        order.append(fake_table.sort_values(by='name')['order'].to_numpy())

    name = sorted(fake_table['name'])
    min_order = np.array(order).min(axis=0)

    table = pd.DataFrame({'order': min_order, 'name': name})
    table = table.sort_values(by='order').reset_index(drop=True)
    
    return table


def make_fake_tables(scores_dir, fake_dir, field, algos):
    """
    Read the score files with fakes. Read the fake descriptions.
    Make the tables with score oderings.
    """
    fake_tables = {}
    for algo in algos:
        scores = np.memmap(os.path.join(scores_dir, 'score_{}_{}_fake.dat'.format(field, algo)), dtype=np.float64)
        fake_names = load_fake_names(os.path.join(fake_dir, 'fakes_{}_fake.csv'.format(field)))
        fake_tables[algo] = fake_indices(scores, fake_names)
    
    fake_tables['union'] = union_fakes(fake_tables)
    return fake_tables

# Generating score files with scalings

In [4]:
# Generate the score files for m31 field with different scalings

field = 'm31'
for scaling in scalings:
    # Put score files in different directories for future comparison
    scores_dir = os.path.join(data_dir, 'scores_' + scaling)
    os.makedirs(scores_dir, exist_ok=True)

    for algo in algos_for_fields[field]:
        real_args = ['--oid', os.path.join(data_dir, 'oid_{}.dat'.format(field)),
                     '--feature', os.path.join(data_dir, 'feature_{}.dat'.format(field)),]
        fake_args = ['--oid', os.path.join(fake_dir, 'oid_{}_fake.dat'.format(field)),
                     '--feature', os.path.join(fake_dir, 'feature_{}_fake.dat'.format(field)),]
        
        score_file = os.path.join(scores_dir, 'score_{}_{}_fake.dat'.format(field, algo))
        score_args = ['--output', score_file]
        
        if os.path.exists(score_file):
            continue
        
        args = ['--jobs', jobs, '--scale', scaling, '--classifier', algo]
        args += real_args + fake_args + score_args
        ZtfAnomalyDetector(args).run()

15,-1778.3646750393225
695211400034403,-480.0181397834178
695211400124577,-387.76408131373086
695211200009221,-374.7629510113264
695211400000352,-371.4710696830313
1,-337.69876158785934
695211400102351,-336.5294059450145
695211400053697,-324.0251410907933
695211200020939,-297.79968850738743
695211200008801,-286.753753171906
695211400028274,-273.4342476704372
695211200015230,-265.5409956767474
695211400133827,-238.01986298914707
695211400088968,-235.01272762341685
695211200008024,-229.06143047432388
10,-218.04961671374767
695211200009296,-214.96797483723165
695211200075348,-213.56608371467874
695211400117334,-210.09442826258348
695211200009492,-209.66619283353023
695211200032218,-207.0146073380352
4,-206.42796564743654
695211200013685,-203.30247091292992
695211200020898,-202.43267528759617
9,-198.60122947755676
695211200054816,-195.2444880503959
695211200042829,-193.3503427034743
695211400053352,-192.5499277980547
695211400134715,-190.84531680665748
2,-189.25218211548147
695211200077003

/home/kostya/.venv/zwad/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


15,-0.7624333489749442
3,-0.7505975075272017
1,-0.7486079465277024
10,-0.7473635451838108
695211400034403,-0.7339690623391737
695211400124577,-0.7330883738948656
4,-0.7283887501551958
695211400102351,-0.7235211017102217
695211400048384,-0.7160566283595363
695211400053697,-0.7145713282469589
695211400066438,-0.7105840677295269
695211400121607,-0.7091121924668289
695211400008325,-0.703335919637042
0,-0.7010709543435328
695211400028274,-0.6991100824629096
695211400053352,-0.698581010502337
695211400050325,-0.6982550021740399
9,-0.6979159210458191
695211100011888,-0.6978123530726846
695211100022045,-0.6960713096546127
695211400025713,-0.6951821868652179
695211400114292,-0.6945100596324468
695211400132769,-0.6933181776510686
695211400088066,-0.6927858115690209
695211400134229,-0.6896752975291597
695211400128356,-0.6876492339648157
695211400134257,-0.6861352139017289
12,-0.685898134248803
695211200082131,-0.6852293525849206
2,-0.6818811085126298
695211400009049,-0.6801895072323626
6952114001

/home/kostya/.venv/zwad/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


1,-0.7746894863748204
4,-0.772249305837574
10,-0.7542577303373378
695211400034403,-0.7459154361942975
0,-0.7434221561569662
9,-0.7400310826449431
695211100002984,-0.7382952536788912
695211100005490,-0.7365106799172852
695211400124577,-0.7288844148951584
3,-0.7265290381667888
695211100011888,-0.7240478352330637
695211200000865,-0.7229099708554823
15,-0.7223220646484859
695211100008614,-0.7134651031499197
695211400000352,-0.7107922875981548
695211100004663,-0.7057546852142185
695211100018408,-0.7047699727156239
695211100021872,-0.7027713637921413
695211400028274,-0.7013688606352377
695211400114292,-0.6991444954001503
695211400053697,-0.6965232031140587
695211200050499,-0.6940005053616143
695211200020939,-0.6923372866265117
695211400134810,-0.6909937048727235
14,-0.6900386074083762
695211400134970,-0.6883793773930159
695211100002099,-0.6853974395028325
695211400070144,-0.6842922963369769
695211400102351,-0.6832004713687397
695211100055146,-0.6816383386123532
695211400133993,-0.68058281530

/home/kostya/.venv/zwad/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


1,-0.7601746041358256
15,-0.7565510136748009
4,-0.7475398721158389
3,-0.7408451889869485
10,-0.7342334153120329
695211400034403,-0.7306582984674507
0,-0.7273180195469725
9,-0.7265457594866601
695211400124577,-0.714402670009327
695211100002984,-0.7121926903939685
695211100005490,-0.7115678834365743
695211100011888,-0.7015160818031062
695211400053697,-0.6988787260315046
695211200000865,-0.695309376069903
695211200009221,-0.6928264268304528
695211200020939,-0.6924288690793181
695211400102351,-0.6881176445992448
695211100008614,-0.6875290411988095
695211200008801,-0.687341812688105
695211400028274,-0.6872503269075201
695211200050499,-0.6855547009962285
695211400051367,-0.6839446595905292
695211100021872,-0.6812313817083269
8,-0.6750821458965078
695211400117334,-0.6746487698886228
695211400121607,-0.6744075547148162
695211100018408,-0.6740805633151459
695211100004663,-0.6730528033932686
2,-0.6728610022550202
695211100127578,-0.6709999127782004
695211400114292,-0.6698955850313096
69521140013

/home/kostya/.venv/zwad/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


13,-0.7322708958646795
15,-0.7138915868763351
1,-0.706554863972854
10,-0.6942930505777014
695211400121607,-0.6752159579594086
695211200075348,-0.6725297076518184
695211200008024,-0.6677676679683469
0,-0.6673973403772633
9,-0.6629533435276785
695211400009049,-0.6599220253266959
2,-0.6552832884602564
8,-0.6538264585958435
695211400027347,-0.6526326503276635
695211200032218,-0.6512303840227942
695211400128515,-0.6497044700892087
3,-0.6496761556396472
695211200009221,-0.6490633260204473
695211200035023,-0.6488785152849066
695211400053697,-0.6484395556817357
695211200020939,-0.6471116694185286
695211400133827,-0.6463992703896871
12,-0.6462073414115677
695211400034403,-0.6452535233886803
695211400088968,-0.6447151269984414
695211100022045,-0.6444716885759069
695211100003383,-0.6444033643761363
4,-0.643758971118233
695211200015230,-0.6410744928700705
695211400070144,-0.6402992899880631
695211200009492,-0.6397462102834847
695211400124577,-0.6370600756686887
695211400126897,-0.6358086354964934


# Compare fake detection with scalings

In [5]:
# Plot the fake curves

field = 'm31'
oid_len = os.stat(os.path.join(data_dir, 'oid_m31.dat')).st_size // 8
fig1, ax = plt.subplots(nrows=len(scalings), sharex=True, figsize=(8, 10))
fig2, bx = plt.subplots(figsize=(8, 3))
for i, scaling in enumerate(scalings):
    scores_dir = os.path.join(data_dir, 'scores_' + scaling)
    fake_tables = make_fake_tables(scores_dir, fake_dir, field, algos_for_fields[field])

    for algo, fake_table in fake_tables.items():
        ax[i].plot(fake_table['order'] + 1, np.arange(len(fake_table)) + 1, label=algo)

    ax[i].set(title='{}, {}'.format(field, scaling), ylabel='# of fakes')
    ax[i].set(xscale='log', xlim=[1, oid_len])
    ax[i].legend(loc='lower right')
    ax[i].grid()
    
    fake_table = fake_tables['union']
    bx.plot(fake_table['order'] + 1, np.arange(len(fake_table)) + 1, label=scaling)
#    display(pd.DataFrame({k: v['name'] for k, v in algo_to_fakes.items()}))

ax[-1].set(xlabel='# of outliers')
fig1.tight_layout()

bx.set(title='m31', ylabel='# of fakes', xlabel='# of outliers', xscale='log')
bx.legend(loc='lower right')
bx.grid()
fig2.tight_layout()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Generating score files for different fields

## Score files may be downloaded
```shell
cd ../data
wget "http://sai.snad.space/ztf/scores.tar.gz" -O - | tar -zxf -
```

## Or they may be generated

In [6]:
# Generate the score files for m31 field with different scalings
# WARNING: that may be a long run

scaling = 'std'
scores_dir = os.path.join(data_dir, 'scores')
os.makedirs(scores_dir, exist_ok=True)

for field in algos_for_fields.keys():
    for algo in algos_for_fields[field]:
        real_args = ['--oid', os.path.join(data_dir, 'oid_{}.dat'.format(field)),
                     '--feature', os.path.join(data_dir, 'feature_{}.dat'.format(field)),]
        fake_args = ['--oid', os.path.join(fake_dir, 'oid_{}_fake.dat'.format(field)),
                     '--feature', os.path.join(fake_dir, 'feature_{}_fake.dat'.format(field)),]

        score_file = os.path.join(scores_dir, 'score_{}_{}_fake.dat'.format(field, algo))
        score_args = ['--output', score_file]

        if os.path.exists(score_file):
            continue

        args = ['--jobs', jobs, '--scale', scaling, '--classifier', algo]
        args += real_args + fake_args + score_args
        ZtfAnomalyDetector(args).run()

/home/kostya/.venv/zwad/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


15,-0.7936880762411544
10,-0.7801865564252805
1,-0.7775530622948109
695211400034403,-0.7427733835061721
695211400124577,-0.7373016239422778
695211400053697,-0.7308817969692152
695211400102351,-0.7276941351884356
9,-0.7258693170812737
695211400132963,-0.7258656317915116
695211400088968,-0.7254754561467931
695211400121607,-0.7239174650731331
695211400028274,-0.7234017401746605
695211200075348,-0.7220998472353515
695211400117334,-0.7217214024500976
695211400133827,-0.7204798869642073
695211400000352,-0.7173920272604501
3,-0.7114025034623276
695211400128515,-0.7095553910475249
695211400134262,-0.7092366756569449
8,-0.7092238636843077
2,-0.7051805895119955
695211400130264,-0.7045936820075338
0,-0.7012936442323255
695211400053352,-0.6982856202362641
695211400009049,-0.6978272314686702
695211400133656,-0.6975078800012136
695211200035023,-0.6971250000932383
695211400027347,-0.6971182004846157
695211400134257,-0.6959680361499674
695211400134417,-0.695528077339125
695211400123032,-0.695525787743

/home/kostya/.venv/zwad/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


795203200009604,-0.7945977496191137
795205400022890,-0.7864983490099828
1,-0.785854081310238
10,-0.7844092444853392
15,-0.783778132531444
0,-0.7778374010662108
795211200035931,-0.7763355164276784
795205400027537,-0.7713580652438637
2,-0.769059938296434
795209200003484,-0.7630246094572869
795204200026512,-0.7624674784891813
795211400021366,-0.7615140917658137
795212100007964,-0.7590074064072458
795206400033919,-0.7574110066238523
795205400027532,-0.7572001406998218
795204100015114,-0.7554099387065546
795211400023360,-0.7551092383068672
795213400008053,-0.7537772971197229
795206400037347,-0.7522459542991035
795209200029270,-0.750785220528083
795205400032594,-0.749801579722772
795212200039202,-0.7494195535287033
795205100007271,-0.7494186357658607
795204200006882,-0.7489852313196629
795212200014189,-0.7473901161005789
795210200042516,-0.7471764920286369
795202400037130,-0.7470162957730757
795206400027989,-0.7463197888012929
795202400024457,-0.7450611225376877
795215300016556,-0.7449440282

/home/kostya/.venv/zwad/lib/python3.6/site-packages/sklearn/ensemble/_iforest.py:248: FutureWarning: 'behaviour' is deprecated in 0.22 and will be removed in 0.24. You should not pass or set this parameter.
  FutureWarning


807208200059506,-0.8198752016790447
807206200014645,-0.8187555157561706
807209400037670,-0.8173762799936186
807206400014916,-0.8146159809375071
807206200004116,-0.8143109600474285
807210100028861,-0.813482826650651
15,-0.8134169729249885
807202400045768,-0.8130174219299756
807211300006190,-0.8125931547081211
807202400056014,-0.8121289394500943
807216100038423,-0.8118661868625459
807202300038681,-0.8114409703369385
807208300016714,-0.8105878318605411
807206300013468,-0.8100295586218517
807206200023036,-0.8092683793996334
807204300037369,-0.8088839109360357
807203300044912,-0.8078718547528786
807214300051120,-0.8078660279498632
807214300010833,-0.8077149484092846
807209300012026,-0.8075733080423888
807214100007080,-0.8073564220621172
807204400014494,-0.807150397378748
807209100042420,-0.8070283749682599
807215200006611,-0.8056721468368301
807210200004045,-0.8054085935830898
807211400009493,-0.8053876216905695
807203400037050,-0.8048289093714652
807203200013118,-0.8045800000220453
8072071

# Plot the fake detection curves for fields

In [7]:
# Plot the fake curves for fields

scores_dir = os.path.join(data_dir, 'scores')
for field in algos_for_fields.keys():
    oid_len = os.stat(os.path.join(data_dir, 'oid_{}.dat'.format(field))).st_size // 8

    fake_tables = make_fake_tables(scores_dir, fake_dir, field, algos_for_fields[field])

    fig, ax = plt.subplots(figsize=(8, 3))
    for algo, fake_table in fake_tables.items():
        ax.plot(fake_table['order'] + 1, np.arange(len(fake_table)) + 1, label=algo)

    ax.set(title=field, ylabel='# of fakes', xlabel='# of outliers')
    ax.set(xscale='log', xlim=[1, oid_len])
    ax.legend(loc='upper left')
    ax.grid()
    fig.tight_layout()
    display(pd.DataFrame({k: v['name'] for k, v in fake_tables.items()}))

<IPython.core.display.Javascript object>

,iso,gmm,svm,lof,union
0,step,step,step,step,step
1,ZTF18abhjrcf_format_r,Gaia16aye_3_format_r,MACHO-6.6696.60_format_R,flat,MACHO-6.6696.60_format_R
2,Gaia16aye_3_format_r,ZTF18abhjrcf_format_r,Gaia16aye_3_format_r,MACHO-6.6696.60_format_R,ZTF18abhjrcf_format_r
3,ZTF18abaqxrt_format_r,MACHO-6.6696.60_format_R,ZTF18abhjrcf_format_r,flat_noise,flat
4,MACHO-6.6696.60_format_B,ZTF18abaqxrt_format_r,ZTF18abaqxrt_format_r,Gaia16aye_2_format_r,Gaia16aye_3_format_r
5,ZTF18aaztjyd_format_r,Gaia16aye_format_r,MACHO-6.6696.60_format_B,MACHO-6.6696.60_format_B,flat_noise
6,Gaia16aye_format_r,MACHO-6.6696.60_format_B,flat,Gaia16aye_3_format_r,Gaia16aye_2_format_r
7,Gaia16aye_2_format_r,Gaia16aye_2_format_r,Gaia16aye_2_format_r,ZTF18abaqxrt_format_r,ZTF18abaqxrt_format_r
8,MACHO-6.6696.60_format_R,flat,Gaia16aye_format_r,ZTF18acskgwu_format_r,MACHO-6.6696.60_format_B
9,ZTF18acskgwu_format_r,flat_noise,ZTF18acskgwu_format_r,ZTF18abhjrcf_format_r,Gaia16aye_format_r


<IPython.core.display.Javascript object>

,iso,gmm,svm,union
0,Gaia16aye_3_format_r,step,step,step
1,ZTF18abhjrcf_format_r,Gaia16aye_2_format_r,Gaia16aye_2_format_r,Gaia16aye_2_format_r
2,step,Gaia16aye_format_r,Gaia16aye_3_format_r,Gaia16aye_3_format_r
3,Gaia16aye_2_format_r,Gaia16aye_3_format_r,MACHO-6.6696.60_format_R,ZTF18abhjrcf_format_r
4,Gaia16aye_format_r,ZTF18abaqxrt_format_r,Gaia16aye_format_r,Gaia16aye_format_r
5,ZTF18acskgwu_format_r,ZTF18abhjrcf_format_r,ZTF18abaqxrt_format_r,MACHO-6.6696.60_format_R
6,ZTF18ablruzq_format_r,MACHO-6.6696.60_format_R,MACHO-6.6696.60_format_B,ZTF18abaqxrt_format_r
7,MACHO-6.6696.60_format_R,OGLE-LMC-CEP-0227_format_I,flat,MACHO-6.6696.60_format_B
8,ZTF18aaztjyd_format_r,ZTF18acskgwu_format_r,ZTF18abhjrcf_format_r,flat
9,ZTF18abaqxrt_format_r,flat,ZTF18acskgwu_format_r,ZTF18acskgwu_format_r


<IPython.core.display.Javascript object>

,iso,gmm,union
0,step,step,step
1,ZTF18abhjrcf_format_r,Gaia16aye_3_format_r,Gaia16aye_3_format_r
2,MACHO-6.6696.60_format_B,Gaia16aye_format_r,ZTF18abhjrcf_format_r
3,Gaia16aye_format_r,ZTF18abhjrcf_format_r,Gaia16aye_format_r
4,MACHO-6.6696.60_format_R,OGLE-LMC-CEP-0227_format_V,MACHO-6.6696.60_format_B
5,Gaia16aye_3_format_r,ZTF18aaztjyd_format_r,OGLE-LMC-CEP-0227_format_V
6,ZTF18aaztjyd_format_r,MACHO-6.6696.60_format_B,ZTF18aaztjyd_format_r
7,ZTF18abaqxrt_format_r,MACHO-6.6696.60_format_R,MACHO-6.6696.60_format_R
8,ZTF18acskgwu_format_r,ZTF18abaqxrt_format_r,ZTF18abaqxrt_format_r
9,Gaia16aye_2_format_r,Gaia16aye_2_format_r,Gaia16aye_2_format_r


# Plot the cumulative score distributions

In [8]:
for field in algos_for_fields:
    algo_to_fakes = {}
    n = len(algos_for_fields[field])
    fig, ax = plt.subplots(n, 1, sharex=True, figsize=(8, 2 * n))
    for i, algo in enumerate(algos_for_fields[field]):
        scores = np.memmap('../data/scores/score_{}_{}_fake.dat'.format(field, algo), dtype=np.float64)
        ax[i].plot(np.arange(len(scores)), np.sort(scores), label=algo)
        ax[i].set(ylabel='object score')
        ax[i].legend(loc='upper left')
        ax[i].grid()

    ax[-1].set(xlabel='object #')
    ax[0].set(title=field)
    ax[0].set(xscale='log', xlim=[1, oid_len])
    fig.tight_layout()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>